# Tuner TCN

## Imports

In [1]:
import os
import pandas as pd
import json

import tensorflow as tf
from keras_tuner import Hyperband
from tensorflow import keras
from tensorflow.keras import layers
from tcn import TCN
from tensorflow.keras.layers import Dense, Input, concatenate
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping

2022-05-21 19:13:02.600531: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


## Data prep

In [2]:
import sys
sys.path.insert(1, '../')

from data_preparation import prepare_x_data, get_Y_labels, reshape_Y, reshape_X

In [3]:
scaler = StandardScaler()

### X data - features

In [4]:
# TRAIN data
x_train_visual = prepare_x_data('../Data/LLDs_video_openface/train',',', 5, scaler)
x_train_audio = prepare_x_data('../Data/LLDs_audio_eGeMAPS/train',';', 2, scaler) 

### Y data - labels

In [5]:
#Y TRAIN labels
y_train = get_Y_labels('../Data/labels_metadata.csv', 60, 164, scaler)
y_train = reshape_Y(y_train,len(x_train_visual),1,1)

## Tuner

In [6]:
def build_model_visual(hp):
    # Define input layer.
    input = Input(shape=(None, 465))

    # Define hidden layer.
    tcn_v = TCN(nb_filters=hp.Int('v_nb_filters', min_value=1, max_value=465, step=1), 
                kernel_size=hp.Choice('v_kernel_size', [1,2,4,8,16,32,64,128]), 
                dilations=(128,256,512,1024), 
                padding=hp.Choice('v_padding', ['causal', 'same']), 
                nb_stacks=hp.Choice('v_nb_stacks', [1,2]), 
                use_batch_norm=False, 
                use_layer_norm=False,
                use_weight_norm=False, 
                use_skip_connections=True, 
                dropout_rate=hp.Choice('v_dropout_rate', [1e-2, 1e-3, 1e-4]), 
                return_sequences=False, 
                input_shape=(None, 465))(input)

    # Define output layer.
    output = Dense(1, activation='linear')(tcn_v)

    # Define optimizer and show summary.
    model = keras.Model(inputs=[input], outputs=[output])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), loss='mse', metrics=['mae'])

    return model

In [7]:
def build_model_audio(hp):
    # Define input layer.
    input = Input(shape=(None, 23))

    # Define hidden layer.
    tcn_a = TCN(nb_filters=hp.Int('a_nb_filters', min_value=1, max_value=23, step=1), 
                kernel_size=hp.Choice('a_kernel_size', [1,2,4,8,16,32,64,128]), 
                dilations=(128,256,512,1024), 
                padding=hp.Choice('a_padding', ['causal', 'same']), 
                nb_stacks=hp.Choice('a_nb_stacks', [1,2]), 
                use_batch_norm=False, 
                use_layer_norm=False, 
                use_weight_norm=False,
                use_skip_connections=True, 
                dropout_rate=hp.Choice('a_dropout_rate', [1e-2, 1e-3, 1e-4]), 
                return_sequences=False, 
                input_shape=(None, 23))(input)


    # Define output layer.
    output = Dense(1, activation='linear')(tcn_a)

    # Define optimizer and show summary.
    model = keras.Model(inputs=[input], outputs=[output])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), loss='mse', metrics=['mae'])

    return model

In [8]:
 # Define tuner. A new tuner is created for every file.
tuner_v = Hyperband(
    hypermodel=build_model_visual, 
    objective='loss', 
    max_epochs=50, 
    factor=3, 
    project_name='TCN_Tuner_Visual'
)

tuner_a = Hyperband(
    hypermodel=build_model_audio, 
    objective='loss', 
    max_epochs=50, 
    factor=3, 
    project_name='TCN_Tuner_Audio'
)

early_stop = EarlyStopping(monitor='loss', min_delta=0, patience=10, mode='min')

2022-05-21 19:13:37.543487: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-21 19:13:37.551538: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-05-21 19:13:37.876864: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:927] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-05-21 19:13:37.881746: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1660 Ti computeCapability: 7.5
coreClock: 1.59GHz coreCount: 24 deviceMemorySize: 6.00GiB deviceMemoryBandwidth: 268.26GiB/s
2022-05-21 19:13:37.881875: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-05-21 19:13:37.884014: I tensorflow/stream_executor/plat

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [9]:
# Tune the model one file at a time.
#Train visual data tuner
index_v = 0
for train_visual in x_train_visual:
    train_visual = reshape_X(train_visual)

    tuner_v.search(train_visual, y_train[index_v], epochs=10, batch_size=len(train_visual), callbacks=[early_stop]) 
    index_v += 1

Trial 128 Complete [00h 00m 13s]
loss: 0.48831021785736084

Best loss So Far: 0.0001990353048313409
Total elapsed time: 00h 36m 07s
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered

In [10]:
# Tune the model one file at a time.
#Train audio data tuner
index_a = 0
for train_audio in x_train_audio:
    train_audio = reshape_X(train_audio)

    tuner_a.search(train_audio, y_train[index_a], epochs=10, batch_size=len(train_audio), callbacks=[early_stop]) 
    index_a += 1

Trial 128 Complete [00h 00m 14s]
loss: 0.0028477618470788

Best loss So Far: 4.421359335538e-06
Total elapsed time: 00h 21m 25s
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorf

In [11]:
#Get df from all tuner searches
def get_hp_df(tunerdir):
    rootdir = tunerdir
    hp_values = []
    #Get learned hyperparameters for every file (trial) and append to a list
    for trial in os.listdir(rootdir):
        pathdir = os.path.join(rootdir, trial)
        filedir = os.path.join(pathdir, 'trial.json')
        
        if os.path.isdir(pathdir): #looking only for trial subfolders
            with open(filedir) as json_file:
                data = json.load(json_file)
                values = data['hyperparameters']['values'] #get learned hyperparameters
                hp_values.append(values) #append to list
        
    #Transform to dataframe
    hp_df = pd.DataFrame(hp_values)

    return hp_df

In [12]:
# Get average hyperparameter values for both modalities
hpdf_multi_v = get_hp_df('../TCN/TCN_Tuner_Visual')
hpdf_multi_a = get_hp_df('../TCN/TCN_Tuner_Audio')

In [13]:
print(""" Average hyperparameters for each data modality.
--------FOR VISUAL DATA:""")
hpdf_multi_v.mean()

 Average hyperparameters for each data modality.
--------FOR VISUAL DATA:


/tmp/ipykernel_6851/2020555760.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  hpdf_multi_v.mean()


v_nb_filters           240.757812
v_kernel_size           30.765625
v_nb_stacks              1.398438
v_dropout_rate           0.003573
learning_rate            0.003869
tuner/epochs            15.078125
tuner/initial_epoch      2.437500
tuner/bracket            2.078125
tuner/round              0.390625
dtype: float64

In [14]:
print(""" Average hyperparameters for each data modality.
--------FOR AUDIO DATA:""")
hpdf_multi_a.mean()

 Average hyperparameters for each data modality.
--------FOR AUDIO DATA:


/tmp/ipykernel_6851/3992919396.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  hpdf_multi_a.mean()


a_nb_filters           11.843750
a_kernel_size          32.750000
a_nb_stacks             1.328125
a_dropout_rate          0.005071
learning_rate           0.004016
tuner/epochs           15.078125
tuner/initial_epoch     2.437500
tuner/bracket           2.078125
tuner/round             0.390625
dtype: float64

: 